In [1]:
import os
import pandas as pd
%pwd

'c:\\Users\\pavan\\Desktop\\Fraud-Detection-E2E-ML-project-using-MLflow\\Notebooks'

In [3]:
os.chdir('../')

In [4]:
data = pd.read_csv('artifacts/data_ingestion/fraud_data.csv')

In [5]:
data.columns

Index(['trans_date_trans_time', 'category', 'amt', 'gender', 'city_pop', 'dob',
       'fraud'],
      dtype='object')

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 7 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   trans_date_trans_time  1048575 non-null  object 
 1   category               1048575 non-null  object 
 2   amt                    1048575 non-null  float64
 3   gender                 1048575 non-null  object 
 4   city_pop               1048575 non-null  int64  
 5   dob                    1048575 non-null  object 
 6   fraud                  1048575 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 56.0+ MB


In [9]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    data_dir: Path
    all_schema: dict

In [10]:
from fraud_detection_project.constants import *
from fraud_detection_project.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    # First setting up the environment by defining the file paths of configuration file, params file and schema file.  
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        # Creating the root directory in the artifacts folder
        create_directories([self.config.artifacts_root])
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            data_dir = config.data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [12]:
import os
from fraud_detection_project import logger

In [13]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [16]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2023-08-23 20:05:32,926: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-23 20:05:32,928: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-23 20:05:32,932: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-23 20:05:32,936: INFO: common: created directory at: artifacts]
[2023-08-23 20:05:32,938: INFO: common: created directory at: artifacts/data_validation]
